#### Import Modules

In [1]:
import numpy as np
import pandas as pd
import sys, os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from read_write_util import ReadWriteUtil
from cleaning_pipeline import CleaningPipeline

In [2]:
warnings.filterwarnings('ignore')
read_write_util = ReadWriteUtil()
# Initialize Read and Write helper

#### Read Data

In [3]:
df_store = read_write_util.dvc_get_data(path="../data/store.csv", version="v1")
df_store.head()

2022-09-06 11:56:10,007:logger:../data/store.csv with version v1 Loaded


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [4]:
df_train = read_write_util.dvc_get_data(path="../data/train.csv", version='v1')
df_train.head()

2022-09-06 11:56:10,545:logger:../data/train.csv with version v1 Loaded


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
df_test = read_write_util.dvc_get_data("../data/test.csv", version='v1')
df_test.head()

2022-09-06 11:56:10,625:logger:../data/test.csv with version v1 Loaded


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


#### Sort and Index

In [6]:
df_test.sort_values(['Store', 'Date'], ignore_index=True, inplace=True)
df_train.sort_values(['Store', 'Date'], ignore_index=True, inplace=True)
# df_test.set_index('Date', inplace=True)
# df_train.set_index('Date', inplace=True)

#### Data Overview

In [7]:
print(f"SHAPE: {df_store.shape}")
df_store.info()

SHAPE: (1115, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [8]:
print(f"SHAPE: {df_train.shape}")
df_train.info()

SHAPE: (1017209, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [9]:
print(f"SHAPE: {df_test.shape}")
df_test.info()

SHAPE: (41088, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             41088 non-null  int64  
 1   Store          41088 non-null  int64  
 2   DayOfWeek      41088 non-null  int64  
 3   Date           41088 non-null  object 
 4   Open           41077 non-null  float64
 5   Promo          41088 non-null  int64  
 6   StateHoliday   41088 non-null  object 
 7   SchoolHoliday  41088 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 2.5+ MB


#### Check for duplicates

`We have two unique values`

In [10]:
df_train.duplicated(subset=['Store']).all()

False

In [11]:
df_test.duplicated(subset=['Store']).all()


False

In [12]:
df_store.duplicated(subset=['Store']).all()

False

#### Basic cleaning using our pipeline

In [13]:
"""
Function descriptions
convert_to_string(self, df, columns): Converts types to string
convert_to_datetime(self, df, columns): Converts types to Datetime
add_custom_column(self, df, column, value): Add a column to our dataframe
"""
# df_train.reset_index()
pipeline = CleaningPipeline()
pipeline.run_pipeline(df_train, date_cols=['Date'], string_cols=['StateHoliday'])
df_train.set_index('Date', inplace=True)


2022-09-06 11:56:13,731:logger:['StateHoliday'] converted to string.
2022-09-06 11:56:13,842:logger:['Date'] converted to datetime.
2022-09-06 11:56:15,819:logger:Column 'Year' added.
2022-09-06 11:56:17,376:logger:Column 'Month' added.
2022-09-06 11:56:18,907:logger:Column 'DayOfMonth' added.
2022-09-06 11:56:20,817:logger:Column 'WeekOfYear' added.
2022-09-06 11:56:21,067:logger:Column 'weekday' added.


#### Preprocess store data 

##### Handle missing values

In [14]:
totalCells = np.product(df_store.shape)
totalMissing = pipeline.percent_missing_values(df_store)
print("The dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")
# Calculate percentage of missing values


2022-09-06 11:56:21,101:logger:2343 missing values found


The dataset contains 21.01 % missing values.


In [15]:
missing_table_df = pipeline.table_for_missing_values(df_store)
missing_table_df
#We can see that we have two relatable group with some pattern/relation
# we can see the missing values in these groups have the same number.

2022-09-06 11:56:21,254:logger:Your selected dataframe has 10 columns.
There are 6 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Promo2SinceWeek,544,48.79,float64
Promo2SinceYear,544,48.79,float64
PromoInterval,544,48.79,object
CompetitionOpenSinceMonth,354,31.75,float64
CompetitionOpenSinceYear,354,31.75,float64
CompetitionDistance,3,0.27,float64


In [16]:
missing_rows = pipeline.count_missing_rows(df_store)
# Calculate total number of rows
total_rows = df_store.shape[0]

# Calculate the percentage of missing rows
print(f"{missing_rows} rows({round(((missing_rows/total_rows) * 100), 2)}%) contain atleast one missing value.")

2022-09-06 11:56:21,455:logger:750 rows have missing values


750 rows(67.26%) contain atleast one missing value.


In [17]:
missing_rows = pipeline.count_missing_rows(df_store[['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']])
# Calculate total number of rows
total_rows = df_store.shape[0]

# Calculate the percentage of missing rows
print(f"{missing_rows} rows({round(((missing_rows/total_rows) * 100), 2)}%) contain atleast one missing value.")

#Rows with missing values for the columns ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

2022-09-06 11:56:21,594:logger:544 rows have missing values


544 rows(48.79%) contain atleast one missing value.


In [18]:
missing_rows = pipeline.count_missing_rows(df_store[['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']])
# Calculate total number of rows
total_rows = df_store.shape[0]

# Calculate the percentage of missing rows
print(f"{missing_rows} rows({round(((missing_rows/total_rows) * 100), 2)}%) contain atleast one missing value.")

#Rows with missing values for the columns ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']
"""
Each columuns in ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'] and ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']
have missing values. 
These means the missing values in these groups appear at the same time as a cluster.The values of the columns 
in each group share a common information.We can further investigate.
"""

2022-09-06 11:56:21,750:logger:354 rows have missing values


354 rows(31.75%) contain atleast one missing value.


"\nEach columuns in ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'] and ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']\nhave missing values. \nThese means the missing values in these groups appear at the same time as a cluster.The values of the columns \nin each group share a common information.We can further investigate.\n"

In [19]:
df_pr2 = df_store[df_store['Promo2SinceWeek'].isna()]
df_pr2.head(10)
#Missing rows for column 'Promo2SinceWeek'
#We can see that the Promo2 column is 0 for the first 10 rows.And We can infer that if there is no promotion the
#values for the columns ['Promo2SinceWeek', 'Promo2SinceYear', and 'PromoInterval'] should be null.

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN
15,16,a,c,3270.0,NaN,NaN,0,NaN,NaN,NaN
22,23,d,a,4060.0,8.0,2005.0,0,NaN,NaN,NaN


In [20]:
pipeline.get_unique_values(df_pr2)

,Column,Unique values
0,Store,"[1, 758, 741, 742, 747, 751, 755, 757, 760, 4,..."
1,StoreType,"[a, d, c, b]"
2,Assortment,"[a, c, b]"
3,CompetitionDistance,"[250.0, 140.0, 50.0, 2640.0, 30.0, 420.0, 720...."
4,CompetitionOpenSinceMonth,"[9.0, 4.0, 11.0, 3.0, 12.0, 10.0, 7.0, 6.0, 2...."
5,CompetitionOpenSinceYear,"[2013.0, 2012.0, 2014.0, 2009.0, 2010.0, 2005...."
6,Promo2,[0]
7,Promo2SinceWeek,[]
8,Promo2SinceYear,[]
9,PromoInterval,[]


In [21]:
df_pr2['Promo2'].value_counts()

0    544
Name: Promo2, dtype: int64

In [22]:
df_store['Promo2'].value_counts()
#All rows with 0 value in their Promo2 columns have null values in the columns 
# ['Promo2SinceWeek', 'Promo2SinceYear', and 'PromoInterval'].
"""
let's impute the missing values in the columns Promo2SinceWeek and Promo2SinceYear with 0 to convey 
the meaning of absence since year and week can't be 0.
For PromoInterval we should dig deeper what kind of data it holds since its data type is object.
"""

"\nlet's impute the missing values in the columns Promo2SinceWeek and Promo2SinceYear with 0 to convey \nthe meaning of absence since year and week can't be 0.\nFor PromoInterval we should dig deeper what kind of data it holds since its data type is object.\n"

In [23]:
pd.api.types.infer_dtype(df_store['PromoInterval'])

'string'

In [24]:
df_store['PromoInterval'].value_counts()
# all the unique values 
"""
For the column PromoInterval, we will impute it with '0,0,0,0' as the other values follow list-type format containing four months. 
So inorder to impute these columns we should first investigate if the values 
we will insert exist in each column as this will oppose our intention.
"""

"\nFor the column PromoInterval, we will impute it with '0,0,0,0' as the other values follow list-type format containing four months. \nSo inorder to impute these columns we should first investigate if the values \nwe will insert exist in each column as this will oppose our intention.\n"

In [25]:
df_store[df_store['Promo2SinceWeek'] == 0].shape

(0, 10)

In [26]:
df_store[df_store['Promo2SinceYear'] == 0].shape

(0, 10)

In [27]:
df_store[df_store['PromoInterval'] == '0,0,0,0'].shape

(0, 10)

In [28]:
pipeline.fix_missing_value(df_store, ['Promo2SinceWeek', 'Promo2SinceYear'], 0)

2022-09-06 11:56:26,061:logger:544 missing values in the column Promo2SinceWeek have been replaced by 0.
2022-09-06 11:56:26,068:logger:544 missing values in the column Promo2SinceYear have been replaced by 0.


In [29]:
pipeline.fix_missing_value(df_store, ['PromoInterval'], '0,0,0,0')


2022-09-06 12:05:52,558:logger:544 missing values in the column PromoInterval have been replaced by 0,0,0,0.


In [30]:
# dataframe containing the missing rows for columns ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']
df_cosm = df_store[df_store['CompetitionOpenSinceMonth'].isna()]
df_cosm.head(10)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
11,12,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
12,13,d,a,310.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
15,16,a,c,3270.0,NaN,NaN,0,0.0,0.0,"0,0,0,0"
18,19,a,c,3240.0,NaN,NaN,1,22.0,2011.0,"Mar,Jun,Sept,Dec"
21,22,a,a,1040.0,NaN,NaN,1,22.0,2012.0,"Jan,Apr,Jul,Oct"
25,26,d,a,2300.0,NaN,NaN,0,0.0,0.0,"0,0,0,0"
28,29,d,c,2170.0,NaN,NaN,0,0.0,0.0,"0,0,0,0"
31,32,a,a,2910.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
39,40,a,a,180.0,NaN,NaN,1,45.0,2009.0,"Feb,May,Aug,Nov"
40,41,d,c,1180.0,NaN,NaN,1,31.0,2013.0,"Jan,Apr,Jul,Oct"


In [32]:
# all unique values for each column in the dataframe
pipeline.get_unique_values(df_cosm)
"""
from the column names and the data fields' description in the
challange document we can understand that if CompetitionOpenSinceMonth and CompetitionOpenSinceYear are null 
it means there was already a competition when the store was opened. So we need impute the missing values in a meaningful way.
Before doing that we will hold this meaning of an already existing competition in a separate column named
CompetitionBeforeStoreOpened so that we can make use of this information in our analysis and prediction.
This column will hold values of 0 and 1, 0 indicating the absence of competition at the time the store was 
opened and 1 indicating the presence of competition at the time the store was opened.
"""

,Column,Unique values
0,Store,"[12, 853, 767, 766, 764, 762, 757, 756, 742, 7..."
1,StoreType,"[a, d, c, b]"
2,Assortment,"[c, a, b]"
3,CompetitionDistance,"[250.0, 2410.0, 50.0, 210.0, 840.0, 220.0, 150..."
4,CompetitionOpenSinceMonth,[]
5,CompetitionOpenSinceYear,[]
6,Promo2,"[1, 0]"
7,Promo2SinceWeek,"[0.0, 40.0, 14.0, 45.0, 31.0, 22.0, 5.0, 1.0, ..."
8,Promo2SinceYear,"[0.0, 2013.0, 2011.0, 2009.0, 2012.0, 2014.0, ..."
9,PromoInterval,"[0,0,0,0, Jan,Apr,Jul,Oct, Feb,May,Aug,Nov, Ma..."


In [34]:
df_store['CompetitionBeforeStoreOpened'] = df_store['CompetitionOpenSinceYear'].apply(lambda x: 1 if np.isnan(x) else 0)
df_store[['CompetitionOpenSinceYear', 'CompetitionBeforeStoreOpened']]
# deriving new column
"""
We can impute the columns CompetitionOpenSinceMonth and CompetitionOpenSinceYear. So we will impute missing values in the
column CompetitionOpenSinceYear and CompetitionOpenSinceMonth with its minimun value and 1 respectively.
This kind of imputing assumes that the stores with missing values in these columns were opened after the first month
of the minimum year value in the column CompetitionOpenSinceYear.

"""

,CompetitionOpenSinceYear,CompetitionBeforeStoreOpened
0,2008.0,0
1,2007.0,0
2,2006.0,0
3,2009.0,0
4,2015.0,0
...,...,...
1110,2014.0,0
1111,2006.0,0
1112,NaN,1
1113,NaN,1


In [36]:
min_year = df_store['CompetitionOpenSinceYear'].min()
min_year
#Find the minimum year value for CompetitionOpenSinceYear and impute the missing values.

1900.0

In [37]:
# impute the column CompetitionOpenSinceYear with 1900 and the column CompetitionOpenSinceMonth with 1
pipeline.fix_missing_value(df_store, ['CompetitionOpenSinceYear'], min_year)
pipeline.fix_missing_value(df_store, ['CompetitionOpenSinceMonth'], 1)

2022-09-06 12:28:41,005:logger:354 missing values in the column CompetitionOpenSinceYear have been replaced by 1900.0.
2022-09-06 12:28:41,008:logger:354 missing values in the column CompetitionOpenSinceMonth have been replaced by 1.


In [38]:
df_store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionBeforeStoreOpened
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,"0,0,0,0",0
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",0
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",0
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,"0,0,0,0",0
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,"0,0,0,0",0


In [39]:
df_temp = df_store[df_store['CompetitionDistance'].isna()]
df_temp

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionBeforeStoreOpened
290,291,d,a,NaN,1.0,1900.0,0,0.0,0.0,"0,0,0,0",1
621,622,a,c,NaN,1.0,1900.0,0,0.0,0.0,"0,0,0,0",1
878,879,d,a,NaN,1.0,1900.0,1,5.0,2013.0,"Feb,May,Aug,Nov",1


In [40]:
pipeline.get_unique_values(df_temp)

,Column,Unique values
0,Store,"[291, 622, 879]"
1,StoreType,"[d, a]"
2,Assortment,"[a, c]"
3,CompetitionDistance,[]
4,CompetitionOpenSinceMonth,[1.0]
5,CompetitionOpenSinceYear,[1900.0]
6,Promo2,"[0, 1]"
7,Promo2SinceWeek,"[0.0, 5.0]"
8,Promo2SinceYear,"[0.0, 2013.0]"
9,PromoInterval,"[0,0,0,0, Feb,May,Aug,Nov]"


In [42]:
max_dist = df_store['CompetitionDistance'].max()
max_dist
#the competitions for these stores are very far away and have almost no impact on these stores that they weren't measured
#when collecting the data. For this reason, 
#We will impute this values with the maximum competition distance.

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionBeforeStoreOpened
290,291,d,a,NaN,1.0,1900.0,0,0.0,0.0,"0,0,0,0",1
621,622,a,c,NaN,1.0,1900.0,0,0.0,0.0,"0,0,0,0",1
878,879,d,a,NaN,1.0,1900.0,1,5.0,2013.0,"Feb,May,Aug,Nov",1
